## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=64
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict


In [6]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)
num_bkgnd=8000
### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
# s_val=np.load(fname,mmap_mode='r')[10000:20000][:,0,:,:]
s_val=np.load(fname,mmap_mode='r')[10000:10000+num_bkgnd][:,0,:,:]

print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(8000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [7]:
df_runs=pd.DataFrame([])

In [8]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:500][:,:,:,0])
dict1['num_imgs']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


In [9]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['run5_fixed_cosmology','run6_fixed_cosmology','run7_no_truncated_normal','run8_no_truncated_normal']
prefix='keras_'

for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key=prefix+strg
    fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:500]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


In [11]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201214_121039_lambda1.0_very_good_models/'
prefix='pytorch_1_'

lst=[(3, 10460), (10, 32880), (10, 33230), (15, 47070), (16, 52700), (17, 55860)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'gen_img_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



(64, 128, 128)
0.98355204 -0.9999976
(64, 128, 128)
0.9853262 -0.997866
(64, 128, 128)
0.98270434 -0.99875444
(64, 128, 128)
0.983614 -0.9975422
(64, 128, 128)
0.98324436 -0.9954721
(64, 128, 128)
0.9836703 -0.9972815


In [12]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201214_121039_lambda1.0_very_good_models/'
prefix='pytorch_2_'

lst=[(3, 10460), (10, 32880), (10, 33230), (15, 47070), (16, 52700), (17, 55860)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'inference_0_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


(1000, 128, 128)
0.9944289 -0.9999998
(1000, 128, 128)
0.99345326 -0.99883777
(1000, 128, 128)
0.9875575 -0.99967635
(1000, 128, 128)
0.98994416 -0.99848044
(1000, 128, 128)
0.9897598 -0.99705565
(1000, 128, 128)
0.9836897 -0.9975221


In [13]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201220_073758_lambda2.0/'
prefix='pytorch_3_'

lst=[(7, 22010), (10, 33550), (11, 35970), (16, 50810), (19, 60320), (25, 79050)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'gen_img_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



(64, 128, 128)
0.98392755 -0.999743
(64, 128, 128)
0.983372 -0.9996928
(64, 128, 128)
0.9834111 -0.9997707
(64, 128, 128)
0.9818723 -0.99815124
(64, 128, 128)
0.9761708 -0.9970999
(64, 128, 128)
0.98125273 -0.9981004


In [14]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201220_073758_lambda2.0/'
prefix='pytorch_4_'

lst=[(7, 22010), (10, 33550), (11, 35970), (16, 50810), (19, 60320), (25, 79050)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'inference_1_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


(1000, 128, 128)
0.99429137 -0.999994
(1000, 128, 128)
0.98931396 -0.99992794
(1000, 128, 128)
0.9872785 -0.9998696
(1000, 128, 128)
0.98622954 -0.99888253
(1000, 128, 128)
0.9895319 -0.9988311
(1000, 128, 128)
0.9900511 -0.99919343


In [15]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_imgs', 'spec_sdev', 'spec_val'],
      dtype='object')

In [16]:

col_list=['label', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','num_imgs','fname']

df_runs=df_runs.reindex(columns=col_list)


In [17]:
df_runs

,label,chi_1,chi_1a,chi_spec1,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,num_imgs,fname
0,raw,0.014575,0.000112,0.001910,19.401890,0.000187,0.014276,0.000020,4.080889,0.112023,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0028445197484184732, 0.0020816260811514454,...","[1.121347563774932, 0.9576931243390049, 0.2568...","[44450.167879114895, 22573.11075528218, 15349....","[87592.048827034, 62024.131669237926, 44898.48...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_0,0.001702,0.000073,0.052795,23.822981,0.000449,0.001180,0.000055,4.072555,5.345283,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003081765924486072, 0.0023479649643570986, ...","[1.1146491994998429, 0.9643402305198981, 0.260...","[42790.708515013386, 21535.157649232824, 14415...","[85906.63125591779, 60940.73459686716, 43971.7...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_1,0.002058,0.000455,0.066858,21.185616,0.000231,0.001373,0.000113,4.171513,5.868547,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0031456442853499706, 0.002349065715521853, ...","[1.1148820753244182, 0.9669777732485703, 0.261...","[44993.59584742486, 22703.883553150186, 16263....","[86439.30616832257, 60426.54691713405, 45079.8...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
3,keras_2,0.003165,0.002139,0.020309,25.792137,0.000352,0.000674,0.001705,4.130929,1.242998,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0032299157726108983, 0.00241667307354839, 0...","[1.1439634352627102, 0.9270007224057393, 0.253...","[41984.635273342275, 22377.328892028356, 14242...","[82519.15650125599, 59058.62442671653, 43854.1...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
4,keras_3,0.003478,0.001618,0.047763,20.922330,0.000507,0.001353,0.001206,4.001797,5.103600,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0029680794378720322, 0.002208163391797882, ...","[1.0973298711470636, 0.9833819747814072, 0.269...","[45458.16732584579, 21884.70881964065, 14530.5...","[88968.60526771784, 62091.84879396162, 45370.3...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,pytorch_1_3-10460,0.072186,0.060559,1.325975,34.228589,0.010710,0.000917,0.036643,17.807877,40.931604,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.015309792814725004, 0.010171332752292235, 0...","[1.1948516995732001, 0.7893226158856754, 0.281...","[61731.12151814038, 47498.506690639755, 30063....","[123719.60105120763, 91998.11142534015, 61309....",64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,pytorch_1_10-32880,0.011479,0.002770,0.062186,28.572158,0.005534,0.003175,0.000667,17.666865,2.393955,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.015408049487664065, 0.009762315783459232, 0...","[1.1037550299027268, 0.961356283715069, 0.2740...","[56658.09613081806, 35386.209028724705, 21672....","[91943.96559918858, 64760.393021354095, 45504....",64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,pytorch_1_10-33230,0.008319,0.003776,0.081664,28.060112,0.003047,0.001496,0.001015,15.901513,6.185477,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.014272595071387688, 0.01055785029608586, 0....","[1.1185425846817756, 0.9379918291712509, 0.278...","[53692.154283831085, 31129.89553436663, 19169....","[84621.94894956052, 58178.55987079143, 42979.5...",64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,pytorch_1_15-47070,0.005052,0.000573,0.371234,27.881496,0.003464,0.001015,0.000415,13.994740,28.711548,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.009953078698253284, 0.006243728410864906, 0...","[1.107500734801686, 0.9737392380931985, 0.2632...","[55029.88520965588, 32389.524056836304, 19289....","[86995.56582066417, 66183.38273661135, 44355.0...",64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
9,pytorch_1_16-52700,0.023286,0.013157,0.368641,28.917725,0.009093,0.001036,0.009808,16.551800,26.034044,"[-1.031746031584782, -0.6161616

### Compare chi-squares

In [18]:
chi_keys=['label','chi_1','chi_spec1', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,chi_1,chi_spec1,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,raw,0.014575,0.001910,500.0,0.000112,0.000187,0.014276,0.000020,4.080889,0.112023,19.401890
1,keras_0,0.001702,0.052795,500.0,0.000073,0.000449,0.001180,0.000055,4.072555,5.345283,23.822981
2,keras_1,0.002058,0.066858,500.0,0.000455,0.000231,0.001373,0.000113,4.171513,5.868547,21.185616
3,keras_2,0.003165,0.020309,500.0,0.002139,0.000352,0.000674,0.001705,4.130929,1.242998,25.792137
4,keras_3,0.003478,0.047763,500.0,0.001618,0.000507,0.001353,0.001206,4.001797,5.103600,20.922330
5,pytorch_1_3-10460,0.072186,1.325975,64.0,0.060559,0.010710,0.000917,0.036643,17.807877,40.931604,34.228589
6,pytorch_1_10-32880,0.011479,0.062186,64.0,0.002770,0.005534,0.003175,0.000667,17.666865,2.393955,28.572158
7,pytorch_1_10-33230,0.008319,0.081664,64.0,0.003776,0.003047,0.001496,0.001015,15.901513,6.185477,28.060112
8,pytorch_1_15-47070,0.005052,0.371234,64.0,0.000573,0.003464,0.001015,0.000415,13.994740,28.711548,27.881496
9,pytorch_1_16-52700,0.023286,0.368641,64.0,0.013157,0.009093,0.001036,0.009808,16.551800,26.034044,28.917725


## Plot

In [19]:
def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')

In [31]:
def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=np.sqrt((yerr2**2)/(y1**2) + (yerr1**2) * (yerr2**2/(yerr1**4)))/np.sqrt(row.num_imgs)
            
            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            
        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [32]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('raw', 'keras_0', 'keras_1', 'keras_2…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [21]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

,label,chi_1,chi_spec1,num_imgs,chi_2,chi_spec2
0,raw,0.014575,0.001910,500.0,0.000020,0.112023
1,keras_0,0.001702,0.052795,500.0,0.000055,5.345283
2,keras_1,0.002058,0.066858,500.0,0.000113,5.868547
3,keras_2,0.003165,0.020309,500.0,0.001705,1.242998
4,keras_3,0.003478,0.047763,500.0,0.001206,5.103600
5,pytorch_1_3-10460,0.072186,1.325975,64.0,0.036643,40.931604
6,pytorch_1_10-32880,0.011479,0.062186,64.0,0.000667,2.393955
7,pytorch_1_10-33230,0.008319,0.081664,64.0,0.001015,6.185477
8,pytorch_1_15-47070,0.005052,0.371234,64.0,0.000415,28.711548
9,pytorch_1_16-52700,0.023286,0.368641,64.0,0.009808,26.034044
